In [2]:
import zipfile
from PIL import Image
import pytesseract
import cv2 as cv
import numpy as np

# loading the face detection classifier
face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')

with zipfile.ZipFile("readonly/small_img.zip","r") as zip_ref:
    zip_ref.extractall("readonly/images/")